# DyGAT-FR: Dynamic Graph Attention Network with Feedback Refinement
## Comprehensive Experiments for Journal Submission

This notebook contains all experiments, evaluations, and artifact generation for the DyGAT-FR paper.

**Paper Title**: DyGAT-FR: Dynamic Graph Attention Network with Feedback Refinement for Incremental Imbalanced Learning

---

### Contents
1. Environment Setup & Installation
2. Dataset Loading & Graph Construction
3. Model Training (Static & Incremental)
4. Ablation Studies
5. Baseline Comparisons
6. Evaluation & Metrics
7. Figure Generation (Publication Quality)
8. Table Generation (LaTeX)
9. Statistical Analysis
10. Export Results

## 1. Environment Setup & Installation

In [ ]:
# Check if running on Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    # Mount Google Drive for persistent storage
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Running locally")

In [ ]:
# Clone the repository
!rm -rf FAA-NET
!git clone https://github.com/arif-foysal/FAA-NET.git
%cd FAA-NET
!git checkout followup

In [ ]:
# Install dependencies (optimized for Colab - uses pre-installed PyTorch)
# Skip PyTorch install since Colab already has it

# Install PyG - will use pre-built wheels matching Colab's PyTorch
!pip install -q torch-geometric

# Install other packages in a single command (faster)
!pip install -q pandas numpy scikit-learn matplotlib seaborn xgboost lightgbm tabulate

# NOTE: torch-scatter and torch-sparse are OPTIONAL
# Basic PyG operations (GATConv, etc.) work without them
# Only uncomment if you get import errors requiring them:
# !pip install -q torch-scatter torch-sparse

In [ ]:
# Verify installation
import torch
import torch_geometric

print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Import all required modules
import os
import sys
import json
import warnings
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_curve, auc, precision_recall_curve,
    f1_score, recall_score, precision_score,
    roc_auc_score, average_precision_score
)
from scipy import stats

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 11

# Set random seeds
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("All imports successful!")

In [ ]:
# Import DyGAT-FR modules
sys.path.insert(0, '.')

from core.dygat_fr import DyGATFR, DyGATFRLoss, DyGATFRTrainer
from core.dygat_fr.modules import (
    GraphFocalModulation, DyGATConv, 
    GraphPrototypeAttention, FeedbackRefinementModule,
    MinorityReplayBuffer
)
from core.dygat_fr.data_loader import (
    TabularToGraphConverter, TemporalGraphSplitter,
    NIDSDataLoader, create_synthetic_graph, compute_graph_statistics
)
from core.dygat_fr.trainer import IncrementalTrainer
from core.dygat_fr.utils import set_seed, compute_metrics

# Also import original FAA-Net for comparison
from core.model import EDANv3
from core.loss import ImbalanceAwareFocalLoss  # Fixed: was ImbalancedFocalLoss

print("DyGAT-FR modules imported successfully!")

In [ ]:
# Setup device and output directories
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Create output directories
OUTPUT_DIR = 'results/journal_experiments'
FIGURES_DIR = f'{OUTPUT_DIR}/figures'
TABLES_DIR = f'{OUTPUT_DIR}/tables'
CHECKPOINTS_DIR = f'{OUTPUT_DIR}/checkpoints'

for d in [OUTPUT_DIR, FIGURES_DIR, TABLES_DIR, CHECKPOINTS_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")

## 2. Dataset Loading & Preprocessing

Using identical preprocessing pipeline as FAA-Net for fair comparison.

In [ ]:
# ============================================================
# PREPROCESSING PIPELINE (Identical to FAA-Net)
# ============================================================
# Features dropped due to high correlation (> 0.95) - same as FAA-Net
DROPPED_FEATURES = [
    'ct_dst_src_ltm', 'ct_ftp_cmd', 'ct_src_dport_ltm', 'ct_srv_dst',
    'dbytes', 'dloss', 'dwin', 'sbytes', 'sloss'
]

def preprocess_unsw_faanet(train_df, test_df):
    """
    Preprocess UNSW-NB15 using identical pipeline as FAA-Net.
    
    Steps:
    1. Drop ID column
    2. Handle '-' in service column
    3. Replace inf with NaN
    4. Median imputation (fit on train only)
    5. Safe label encoding for categoricals
    6. Drop high-correlation features
    7. StandardScaler (fit on train only)
    
    Returns:
        X_train, X_test, y_train, y_test (numpy arrays)
    """
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # 1. Drop ID column
    for df in [train_df, test_df]:
        if 'id' in df.columns:
            df.drop('id', axis=1, inplace=True)
    
    # 2. Handle '-' in service column
    for df in [train_df, test_df]:
        if 'service' in df.columns:
            df['service'] = df['service'].replace('-', 'none')
    
    # 3. Replace infinite values with NaN
    for df in [train_df, test_df]:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
    
    # 4. Median imputation - fit on train only
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns
    train_medians = train_df[numeric_cols].median()
    train_df[numeric_cols] = train_df[numeric_cols].fillna(train_medians)
    test_df[numeric_cols] = test_df[numeric_cols].fillna(train_medians)  # Use train medians!
    
    # 5. Label encoding for categorical features (safe for unseen labels)
    categorical_features = ['proto', 'service', 'state']
    
    def safe_transform(encoder, series):
        """Handle unseen labels by mapping to 0."""
        mapping = {label: idx for idx, label in enumerate(encoder.classes_)}
        return series.astype(str).map(mapping).fillna(0).astype(int)
    
    for col in categorical_features:
        if col in train_df.columns:
            le = LabelEncoder()
            le.fit(train_df[col].astype(str))
            train_df[col] = le.transform(train_df[col].astype(str))
            test_df[col] = safe_transform(le, test_df[col])
    
    # 6. Drop high-correlation features (same as FAA-Net)
    train_df.drop(columns=DROPPED_FEATURES, errors='ignore', inplace=True)
    test_df.drop(columns=DROPPED_FEATURES, errors='ignore', inplace=True)
    
    # 7. Separate features and labels
    drop_cols = ['label', 'attack_cat']
    
    X_train = train_df.drop(columns=drop_cols, errors='ignore')
    y_train = train_df['label'].values if 'label' in train_df.columns else (train_df['attack_cat'] != 'Normal').astype(int).values
    
    X_test = test_df.drop(columns=drop_cols, errors='ignore')
    y_test = test_df['label'].values if 'label' in test_df.columns else (test_df['attack_cat'] != 'Normal').astype(int).values
    
    # Store feature names
    feature_names = list(X_train.columns)
    
    # 8. StandardScaler - fit on train only
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.values)
    X_test_scaled = scaler.transform(X_test.values)
    
    print(f"Preprocessing complete (FAA-Net pipeline):")
    print(f"  Train shape: {X_train_scaled.shape}")
    print(f"  Test shape: {X_test_scaled.shape}")
    print(f"  Features: {len(feature_names)}")
    print(f"  Dropped features: {DROPPED_FEATURES}")
    print(f"  Train class distribution: Normal={np.sum(y_train==0):,}, Attack={np.sum(y_train==1):,}")
    print(f"  Test class distribution: Normal={np.sum(y_test==0):,}, Attack={np.sum(y_test==1):,}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, feature_names, scaler

print("Preprocessing functions defined (identical to FAA-Net).")

In [ ]:
# Download UNSW-NB15 dataset if not present
# You can also upload manually to Colab

TRAIN_FILE = 'UNSW_NB15_training-set.csv'
TEST_FILE = 'UNSW_NB15_testing-set.csv'

# Check if files exist
if not os.path.exists(TRAIN_FILE):
    print("Dataset not found. Please upload UNSW-NB15 dataset files.")
    print("Expected files: UNSW_NB15_training-set.csv, UNSW_NB15_testing-set.csv")
    
    if IN_COLAB:
        from google.colab import files
        print("\nUpload the dataset files:")
        uploaded = files.upload()
else:
    print(f"Dataset found: {TRAIN_FILE}")

In [ ]:
# Option: Use synthetic data for testing (comment out if using real data)
USE_SYNTHETIC = True  # Set to False to use UNSW-NB15

if USE_SYNTHETIC:
    print("Using synthetic dataset for demonstration...")
    
    # Create larger synthetic dataset
    train_data = create_synthetic_graph(
        n_nodes=10000,
        n_features=33,
        minority_ratio=0.15,
        k_neighbors=10,
        random_state=SEED
    )
    
    test_data = create_synthetic_graph(
        n_nodes=3000,
        n_features=33,
        minority_ratio=0.20,  # Different ratio to simulate drift
        k_neighbors=10,
        random_state=SEED + 1
    )
    
    # Create train/val masks
    n_train = train_data.num_nodes
    perm = torch.randperm(n_train)
    train_data.train_mask = torch.zeros(n_train, dtype=torch.bool)
    train_data.val_mask = torch.zeros(n_train, dtype=torch.bool)
    train_data.train_mask[perm[:int(0.8*n_train)]] = True
    train_data.val_mask[perm[int(0.8*n_train):]] = True
    
else:
    print("Loading UNSW-NB15 dataset with FAA-Net preprocessing...")
    
    # Load raw CSVs
    train_df = pd.read_csv(TRAIN_FILE)
    test_df = pd.read_csv(TEST_FILE)
    
    print(f"Raw train shape: {train_df.shape}")
    print(f"Raw test shape: {test_df.shape}")
    
    # Apply FAA-Net preprocessing pipeline
    X_train, X_test, y_train, y_test, feature_names, scaler = preprocess_unsw_faanet(train_df, test_df)
    
    # Convert to graph format using TabularToGraphConverter
    print("\nConverting to graph format...")
    graph_converter = TabularToGraphConverter(k_neighbors=10, metric='euclidean')
    
    # Fit on training data, transform both
    graph_converter.fit(X_train)
    train_data = graph_converter.transform(X_train, y_train)
    test_data = graph_converter.transform(X_test, y_test)
    
    # Create train/val split
    n_train = train_data.num_nodes
    perm = torch.randperm(n_train)
    train_data.train_mask = torch.zeros(n_train, dtype=torch.bool)
    train_data.val_mask = torch.zeros(n_train, dtype=torch.bool)
    train_data.train_mask[perm[:int(0.8*n_train)]] = True
    train_data.val_mask[perm[int(0.8*n_train):]] = True
    
    print(f"Graph conversion complete.")
    print(f"  Train nodes: {train_data.num_nodes:,}, edges: {train_data.edge_index.size(1):,}")
    print(f"  Test nodes: {test_data.num_nodes:,}, edges: {test_data.edge_index.size(1):,}")

In [ ]:
# Print dataset statistics
print("=" * 60)
print("DATASET STATISTICS")
print("=" * 60)

for name, data in [('Training', train_data), ('Test', test_data)]:
    stats = compute_graph_statistics(data)
    print(f"\n{name} Set:")
    print(f"  Nodes: {stats['num_nodes']:,}")
    print(f"  Edges: {stats['num_edges']:,}")
    print(f"  Features: {stats['num_features']}")
    print(f"  Avg Degree: {stats['avg_degree']:.2f}")
    print(f"  Class Distribution: {stats['class_distribution']}")
    if 'imbalance_ratio' in stats:
        print(f"  Imbalance Ratio: {stats['imbalance_ratio']:.4f}")

In [ ]:
# Create temporal increments for incremental learning experiments
N_INCREMENTS = 5

splitter = TemporalGraphSplitter(
    n_increments=N_INCREMENTS,
    minority_drift=True,
    drift_intensity=0.3
)

increments = splitter.split(train_data, random_state=SEED)

print(f"\nCreated {len(increments)} temporal increments:")
for i, inc in enumerate(increments):
    minority_count = (inc.y == 1).sum().item()
    majority_count = (inc.y == 0).sum().item()
    print(f"  Increment {i+1}: {inc.num_nodes} nodes "
          f"(Minority: {minority_count}, Majority: {majority_count})")

## 3. Model Training

In [ ]:
# Model hyperparameters
MODEL_CONFIG = {
    'in_channels': train_data.x.size(1),
    'hidden_channels': 128,
    'out_channels': 64,
    'num_classes': 2,
    'num_layers': 3,
    'heads': 4,
    'n_prototypes': 8,
    'focal_alpha': 0.25,
    'focal_gamma': 2.0,
    'dropout': 0.3,
    'prototype_momentum': 0.9,
    'use_feedback': True
}

TRAINING_CONFIG = {
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'epochs': 100,
    'curriculum_epochs': 10,
    'early_stopping_patience': 15
}

print("Model Configuration:")
for k, v in MODEL_CONFIG.items():
    print(f"  {k}: {v}")

print("\nTraining Configuration:")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")

In [ ]:
# Create DyGAT-FR model
model = DyGATFR(**MODEL_CONFIG).to(DEVICE)

print(f"\nModel Parameters: {model.count_parameters():,}")
print(f"\nModel Architecture:")
print(model)

In [ ]:
# Create loss function and trainer
loss_fn = DyGATFRLoss(
    focal_alpha=MODEL_CONFIG['focal_alpha'],
    focal_gamma=MODEL_CONFIG['focal_gamma'],
    contrastive_weight=0.1,
    replay_weight=0.1,
    num_classes=2
)

trainer = DyGATFRTrainer(
    model=model,
    loss_fn=loss_fn,
    device=DEVICE,
    lr=TRAINING_CONFIG['lr'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    curriculum_epochs=TRAINING_CONFIG['curriculum_epochs']
)

print("Trainer initialized!")

In [ ]:
# Train DyGAT-FR on full training data (static setting)
print("=" * 60)
print("TRAINING DyGAT-FR (Static Setting)")
print("=" * 60)

history = trainer.train_increment(
    train_data,
    epochs=TRAINING_CONFIG['epochs'],
    train_mask=train_data.train_mask,
    val_mask=train_data.val_mask,
    verbose=True,
    early_stopping_patience=TRAINING_CONFIG['early_stopping_patience']
)

print("\nTraining completed!")

In [ ]:
# Save static model checkpoint
static_checkpoint_path = f"{CHECKPOINTS_DIR}/dygat_fr_static.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'config': MODEL_CONFIG,
    'history': history
}, static_checkpoint_path)
print(f"Static model saved to {static_checkpoint_path}")

### 3.1 Incremental Learning Training

In [ ]:
# Train DyGAT-FR with incremental learning
print("=" * 60)
print("TRAINING DyGAT-FR (Incremental Setting)")
print("=" * 60)

# Create fresh model for incremental training
model_incremental = DyGATFR(**MODEL_CONFIG).to(DEVICE)

incremental_trainer = IncrementalTrainer(
    model=model_incremental,
    loss_fn=loss_fn,
    device=DEVICE,
    lr=TRAINING_CONFIG['lr'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    curriculum_epochs=TRAINING_CONFIG['curriculum_epochs'] // 2
)

# Train on increments
incremental_results = incremental_trainer.train_increments(
    increments,
    epochs_per_increment=TRAINING_CONFIG['epochs'] // N_INCREMENTS,
    verbose=True
)

print("\nIncremental training completed!")

In [ ]:
# Save incremental model checkpoint
incremental_checkpoint_path = f"{CHECKPOINTS_DIR}/dygat_fr_incremental.pt"
torch.save({
    'model_state_dict': model_incremental.state_dict(),
    'config': MODEL_CONFIG,
    'incremental_results': incremental_results
}, incremental_checkpoint_path)
print(f"Incremental model saved to {incremental_checkpoint_path}")

## 4. Ablation Studies

In [ ]:
# Define ablation configurations
ABLATION_CONFIGS = {
    'DyGAT-FR (Full)': MODEL_CONFIG.copy(),
    
    'w/o Feedback': {**MODEL_CONFIG, 'use_feedback': False},
    
    'w/o Prototypes': {**MODEL_CONFIG, 'n_prototypes': 0},
    
    'w/o Focal Mod': {**MODEL_CONFIG, 'focal_alpha': 0.0},
    
    'Reduced Heads (2)': {**MODEL_CONFIG, 'heads': 2},
    
    'Shallow (2 layers)': {**MODEL_CONFIG, 'num_layers': 2},
}

print(f"Running {len(ABLATION_CONFIGS)} ablation configurations...")

In [ ]:
# Run ablation experiments
ablation_results = {}

for name, config in ABLATION_CONFIGS.items():
    print(f"\n{'='*60}")
    print(f"Ablation: {name}")
    print(f"{'='*60}")
    
    # Handle special case for no prototypes
    if config.get('n_prototypes', 8) == 0:
        config['n_prototypes'] = 1  # Minimum required
    
    # Create model
    abl_model = DyGATFR(**config).to(DEVICE)
    
    # Create trainer
    abl_trainer = DyGATFRTrainer(
        model=abl_model,
        loss_fn=loss_fn,
        device=DEVICE,
        lr=TRAINING_CONFIG['lr'],
        weight_decay=TRAINING_CONFIG['weight_decay']
    )
    
    # Train (reduced epochs for ablation)
    abl_history = abl_trainer.train_increment(
        train_data,
        epochs=50,  # Reduced for ablation
        train_mask=train_data.train_mask,
        val_mask=train_data.val_mask,
        verbose=False,
        early_stopping_patience=10
    )
    
    # Evaluate
    metrics = abl_trainer.evaluate(test_data)
    ablation_results[name] = {
        'config': config,
        'history': abl_history,
        'metrics': metrics,
        'params': abl_model.count_parameters()
    }
    
    print(f"  F1: {metrics['f1']:.4f} | Recall: {metrics['recall']:.4f} | "
          f"Precision: {metrics['precision']:.4f} | AUC: {metrics.get('auc', 0):.4f}")

print("\nAblation studies completed!")

## 5. Baseline Comparisons

In [ ]:
# Baseline 1: Standard GAT (without focal modulation)
from torch_geometric.nn import GATConv

class BaselineGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout=0.3):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels // heads, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels, hidden_channels // heads, heads=heads, dropout=dropout)
        self.conv3 = GATConv(hidden_channels, out_channels // heads, heads=heads, dropout=dropout)
        self.classifier = torch.nn.Linear(out_channels, 1)
        self.dropout = dropout
    
    def forward(self, x, edge_index):
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = torch.nn.functional.gelu(self.conv1(x, edge_index))
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = torch.nn.functional.gelu(self.conv2(x, edge_index))
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = self.conv3(x, edge_index)
        return self.classifier(x)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

print("Baseline GAT defined.")

In [ ]:
# Baseline 2: XGBoost and LightGBM (Tree-based)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

def train_tree_baseline(model, X_train, y_train, X_test, y_test):
    """Train and evaluate tree-based baseline."""
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'accuracy': (y_pred == y_test).mean(),
        'f1': f1_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_prob),
        'avg_precision': average_precision_score(y_test, y_prob)
    }
    
    return metrics, y_pred, y_prob

print("Tree baseline functions defined.")

In [ ]:
# Run all baselines
baseline_results = {}

# Prepare data for tree baselines
X_train_np = train_data.x.numpy()
y_train_np = train_data.y.numpy()
X_test_np = test_data.x.numpy()
y_test_np = test_data.y.numpy()

# XGBoost
print("Training XGBoost...")
xgb = XGBClassifier(
    n_estimators=100, max_depth=6, learning_rate=0.1,
    scale_pos_weight=len(y_train_np[y_train_np==0]) / len(y_train_np[y_train_np==1]),
    random_state=SEED, verbosity=0
)
xgb_metrics, xgb_pred, xgb_prob = train_tree_baseline(
    xgb, X_train_np, y_train_np, X_test_np, y_test_np
)
baseline_results['XGBoost'] = {'metrics': xgb_metrics, 'pred': xgb_pred, 'prob': xgb_prob}
print(f"  XGBoost - F1: {xgb_metrics['f1']:.4f}, Recall: {xgb_metrics['recall']:.4f}")

# LightGBM
print("Training LightGBM...")
lgbm = LGBMClassifier(
    n_estimators=100, max_depth=6, learning_rate=0.1,
    is_unbalance=True, random_state=SEED, verbosity=-1
)
lgbm_metrics, lgbm_pred, lgbm_prob = train_tree_baseline(
    lgbm, X_train_np, y_train_np, X_test_np, y_test_np
)
baseline_results['LightGBM'] = {'metrics': lgbm_metrics, 'pred': lgbm_pred, 'prob': lgbm_prob}
print(f"  LightGBM - F1: {lgbm_metrics['f1']:.4f}, Recall: {lgbm_metrics['recall']:.4f}")

# Standard GAT
print("Training Standard GAT...")
gat_model = BaselineGAT(
    in_channels=train_data.x.size(1),
    hidden_channels=128,
    out_channels=64,
    heads=4,
    dropout=0.3
).to(DEVICE)

gat_optimizer = torch.optim.AdamW(gat_model.parameters(), lr=1e-3)
gat_criterion = torch.nn.BCEWithLogitsLoss()

# Training loop for GAT
x_gpu = train_data.x.to(DEVICE)
edge_gpu = train_data.edge_index.to(DEVICE)
y_gpu = train_data.y.float().to(DEVICE)

gat_model.train()
for epoch in range(50):
    gat_optimizer.zero_grad()
    out = gat_model(x_gpu, edge_gpu).squeeze()
    loss = gat_criterion(out[train_data.train_mask], y_gpu[train_data.train_mask])
    loss.backward()
    gat_optimizer.step()

# Evaluate GAT
gat_model.eval()
with torch.no_grad():
    x_test_gpu = test_data.x.to(DEVICE)
    edge_test_gpu = test_data.edge_index.to(DEVICE)
    gat_logits = gat_model(x_test_gpu, edge_test_gpu).squeeze()
    gat_prob = torch.sigmoid(gat_logits).cpu().numpy()
    gat_pred = (gat_prob > 0.5).astype(int)

gat_metrics = {
    'accuracy': (gat_pred == y_test_np).mean(),
    'f1': f1_score(y_test_np, gat_pred),
    'recall': recall_score(y_test_np, gat_pred),
    'precision': precision_score(y_test_np, gat_pred),
    'auc': roc_auc_score(y_test_np, gat_prob),
    'avg_precision': average_precision_score(y_test_np, gat_prob)
}
baseline_results['Standard GAT'] = {'metrics': gat_metrics, 'pred': gat_pred, 'prob': gat_prob}
print(f"  Standard GAT - F1: {gat_metrics['f1']:.4f}, Recall: {gat_metrics['recall']:.4f}")

print("\nAll baselines completed!")

## 6. Evaluation & Metrics

In [ ]:
# Evaluate DyGAT-FR on test set
print("=" * 60)
print("FINAL EVALUATION ON TEST SET")
print("=" * 60)

# Static DyGAT-FR
model.eval()
with torch.no_grad():
    x_test = test_data.x.to(DEVICE)
    edge_test = test_data.edge_index.to(DEVICE)
    logits = model(x_test, edge_test)
    dygat_prob = torch.sigmoid(logits).cpu().numpy().flatten()
    dygat_pred = (dygat_prob > 0.5).astype(int)

dygat_metrics = {
    'accuracy': (dygat_pred == y_test_np).mean(),
    'f1': f1_score(y_test_np, dygat_pred),
    'recall': recall_score(y_test_np, dygat_pred),
    'precision': precision_score(y_test_np, dygat_pred),
    'auc': roc_auc_score(y_test_np, dygat_prob),
    'avg_precision': average_precision_score(y_test_np, dygat_prob)
}

# Store for comparison
all_results = {
    'DyGAT-FR (Static)': {'metrics': dygat_metrics, 'pred': dygat_pred, 'prob': dygat_prob},
    **baseline_results
}

# Print comparison table
print("\n" + "="*80)
print(f"{'Model':<20} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} {'F1':>10} {'AUC':>10}")
print("="*80)

for name, result in all_results.items():
    m = result['metrics']
    print(f"{name:<20} {m['accuracy']:>10.4f} {m['precision']:>10.4f} "
          f"{m['recall']:>10.4f} {m['f1']:>10.4f} {m['auc']:>10.4f}")

print("="*80)

In [ ]:
# Compute confusion matrices
confusion_matrices = {}

for name, result in all_results.items():
    cm = confusion_matrix(y_test_np, result['pred'])
    confusion_matrices[name] = cm
    
    tn, fp, fn, tp = cm.ravel()
    print(f"\n{name}:")
    print(f"  TP: {tp:,} | FP: {fp:,}")
    print(f"  FN: {fn:,} | TN: {tn:,}")
    print(f"  FPR: {fp/(fp+tn):.4f} | FNR: {fn/(fn+tp):.4f}")

In [ ]:
# Incremental learning evaluation - forgetting analysis
print("\n" + "="*60)
print("INCREMENTAL LEARNING - FORGETTING ANALYSIS")
print("="*60)

# Evaluate incremental model on each increment
model_incremental.eval()
forgetting_metrics = []

for i, inc in enumerate(increments):
    with torch.no_grad():
        x_inc = inc.x.to(DEVICE)
        edge_inc = inc.edge_index.to(DEVICE)
        logits = model_incremental(x_inc, edge_inc)
        prob = torch.sigmoid(logits).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
    
    y_inc = inc.y.numpy()
    f1 = f1_score(y_inc, pred)
    recall = recall_score(y_inc, pred)
    
    forgetting_metrics.append({
        'increment': i + 1,
        'f1': f1,
        'recall': recall
    })
    
    print(f"Increment {i+1}: F1={f1:.4f}, Recall={recall:.4f}")

# Calculate forgetting
f1_values = [m['f1'] for m in forgetting_metrics]
forgetting = max(f1_values) - f1_values[-1]
print(f"\nForgetting (max F1 - final F1): {forgetting:.4f} ({forgetting*100:.2f}%)")

## 7. Figure Generation (Publication Quality)

In [ ]:
# Figure 1: Training Curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
ax = axes[0]
ax.plot(history['train_loss'], label='Train', color='#2196F3', linewidth=2)
if 'val_loss' in history:
    ax.plot(history['val_loss'], label='Validation', color='#FF5722', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('(a) Training Loss')
ax.legend()
ax.grid(True, alpha=0.3)

# F1 Score
ax = axes[1]
ax.plot(history['train_f1'], label='Train', color='#2196F3', linewidth=2)
if 'val_f1' in history:
    ax.plot(history['val_f1'], label='Validation', color='#FF5722', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('F1 Score')
ax.set_title('(b) F1 Score')
ax.legend()
ax.grid(True, alpha=0.3)

# Recall
ax = axes[2]
ax.plot(history['train_recall'], label='Train', color='#2196F3', linewidth=2)
if 'val_recall' in history:
    ax.plot(history['val_recall'], label='Validation', color='#FF5722', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Recall')
ax.set_title('(c) Recall')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/fig1_training_curves.pdf', bbox_inches='tight')
plt.savefig(f'{FIGURES_DIR}/fig1_training_curves.png', bbox_inches='tight', dpi=300)
plt.show()
print(f"Saved: {FIGURES_DIR}/fig1_training_curves.pdf")

In [ ]:
# Figure 2: ROC Curves Comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

colors = ['#2196F3', '#4CAF50', '#FF5722', '#9C27B0', '#795548']

# ROC Curves
ax = axes[0]
for i, (name, result) in enumerate(all_results.items()):
    fpr, tpr, _ = roc_curve(y_test_np, result['prob'])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.3f})", 
            color=colors[i % len(colors)], linewidth=2)

ax.plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('(a) ROC Curves')
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, alpha=0.3)

# PR Curves
ax = axes[1]
for i, (name, result) in enumerate(all_results.items()):
    precision_vals, recall_vals, _ = precision_recall_curve(y_test_np, result['prob'])
    ap = average_precision_score(y_test_np, result['prob'])
    ax.plot(recall_vals, precision_vals, label=f"{name} (AP={ap:.3f})",
            color=colors[i % len(colors)], linewidth=2)

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('(b) Precision-Recall Curves')
ax.legend(loc='lower left', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/fig2_roc_pr_curves.pdf', bbox_inches='tight')
plt.savefig(f'{FIGURES_DIR}/fig2_roc_pr_curves.png', bbox_inches='tight', dpi=300)
plt.show()
print(f"Saved: {FIGURES_DIR}/fig2_roc_pr_curves.pdf")

In [ ]:
# Figure 3: Confusion Matrices
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

class_names = ['Normal', 'Attack']
selected_models = ['DyGAT-FR (Static)', 'Standard GAT', 'XGBoost', 'LightGBM']

for ax, name in zip(axes, selected_models):
    cm = confusion_matrices[name]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={'size': 12})
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title(name)

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/fig3_confusion_matrices.pdf', bbox_inches='tight')
plt.savefig(f'{FIGURES_DIR}/fig3_confusion_matrices.png', bbox_inches='tight', dpi=300)
plt.show()
print(f"Saved: {FIGURES_DIR}/fig3_confusion_matrices.pdf")

In [ ]:
# Figure 4: Ablation Study Bar Chart
fig, ax = plt.subplots(figsize=(10, 6))

ablation_names = list(ablation_results.keys())
f1_scores = [ablation_results[name]['metrics']['f1'] for name in ablation_names]
recall_scores = [ablation_results[name]['metrics']['recall'] for name in ablation_names]

x = np.arange(len(ablation_names))
width = 0.35

bars1 = ax.bar(x - width/2, f1_scores, width, label='F1 Score', color='#2196F3')
bars2 = ax.bar(x + width/2, recall_scores, width, label='Recall', color='#4CAF50')

ax.set_ylabel('Score')
ax.set_title('Ablation Study Results')
ax.set_xticks(x)
ax.set_xticklabels(ablation_names, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1.1)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/fig4_ablation_study.pdf', bbox_inches='tight')
plt.savefig(f'{FIGURES_DIR}/fig4_ablation_study.png', bbox_inches='tight', dpi=300)
plt.show()
print(f"Saved: {FIGURES_DIR}/fig4_ablation_study.pdf")

In [ ]:
# Figure 5: Incremental Learning Performance
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

increments_x = [m['increment'] for m in forgetting_metrics]
f1_vals = [m['f1'] for m in forgetting_metrics]
recall_vals = [m['recall'] for m in forgetting_metrics]

# F1 across increments
ax = axes[0]
ax.plot(increments_x, f1_vals, 'o-', color='#2196F3', linewidth=2, markersize=10)
ax.fill_between(increments_x, f1_vals, alpha=0.2, color='#2196F3')
ax.set_xlabel('Increment')
ax.set_ylabel('F1 Score')
ax.set_title('(a) F1 Score Across Increments')
ax.set_xticks(increments_x)
ax.grid(True, alpha=0.3)

for i, (inc, f1) in enumerate(zip(increments_x, f1_vals)):
    ax.annotate(f'{f1:.3f}', (inc, f1), textcoords="offset points",
                xytext=(0, 10), ha='center', fontsize=10)

# Recall across increments
ax = axes[1]
ax.plot(increments_x, recall_vals, 'o-', color='#4CAF50', linewidth=2, markersize=10)
ax.fill_between(increments_x, recall_vals, alpha=0.2, color='#4CAF50')
ax.set_xlabel('Increment')
ax.set_ylabel('Recall')
ax.set_title('(b) Recall Across Increments')
ax.set_xticks(increments_x)
ax.grid(True, alpha=0.3)

for i, (inc, rec) in enumerate(zip(increments_x, recall_vals)):
    ax.annotate(f'{rec:.3f}', (inc, rec), textcoords="offset points",
                xytext=(0, 10), ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/fig5_incremental_performance.pdf', bbox_inches='tight')
plt.savefig(f'{FIGURES_DIR}/fig5_incremental_performance.png', bbox_inches='tight', dpi=300)
plt.show()
print(f"Saved: {FIGURES_DIR}/fig5_incremental_performance.pdf")

In [ ]:
# Figure 6: Model Comparison Bar Chart
fig, ax = plt.subplots(figsize=(12, 6))

models = list(all_results.keys())
metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
x = np.arange(len(models))
width = 0.15

colors = ['#1976D2', '#388E3C', '#F57C00', '#7B1FA2', '#C62828']

for i, metric in enumerate(metrics_to_plot):
    values = [all_results[m]['metrics'][metric] for m in models]
    bars = ax.bar(x + i*width, values, width, label=metric.upper(), color=colors[i])

ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x + width * 2)
ax.set_xticklabels(models, rotation=15, ha='right')
ax.legend(loc='upper right', ncol=5)
ax.set_ylim(0, 1.15)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/fig6_model_comparison.pdf', bbox_inches='tight')
plt.savefig(f'{FIGURES_DIR}/fig6_model_comparison.png', bbox_inches='tight', dpi=300)
plt.show()
print(f"Saved: {FIGURES_DIR}/fig6_model_comparison.pdf")

## 8. Table Generation (LaTeX)

In [ ]:
# Table 1: Main Results Comparison
def generate_latex_table(results_dict, caption, label):
    """Generate LaTeX table from results."""
    latex = []
    latex.append(r"\begin{table}[htbp]")
    latex.append(r"\centering")
    latex.append(r"\caption{" + caption + r"}")
    latex.append(r"\label{" + label + r"}")
    latex.append(r"\begin{tabular}{lccccc}")
    latex.append(r"\toprule")
    latex.append(r"Model & Accuracy & Precision & Recall & F1 & AUC \\")
    latex.append(r"\midrule")
    
    # Find best values for highlighting
    best_f1 = max(r['metrics']['f1'] for r in results_dict.values())
    best_recall = max(r['metrics']['recall'] for r in results_dict.values())
    
    for name, result in results_dict.items():
        m = result['metrics']
        # Bold best F1 and Recall
        f1_str = f"\\textbf{{{m['f1']:.4f}}}" if m['f1'] == best_f1 else f"{m['f1']:.4f}"
        recall_str = f"\\textbf{{{m['recall']:.4f}}}" if m['recall'] == best_recall else f"{m['recall']:.4f}"
        
        latex.append(f"{name} & {m['accuracy']:.4f} & {m['precision']:.4f} & "
                    f"{recall_str} & {f1_str} & {m['auc']:.4f} \\\\")
    
    latex.append(r"\bottomrule")
    latex.append(r"\end{tabular}")
    latex.append(r"\end{table}")
    
    return "\n".join(latex)

# Generate main results table
table1_latex = generate_latex_table(
    all_results,
    "Comparison of DyGAT-FR with baseline methods on test set.",
    "tab:main_results"
)

print("Table 1: Main Results")
print("=" * 60)
print(table1_latex)

# Save to file
with open(f'{TABLES_DIR}/table1_main_results.tex', 'w') as f:
    f.write(table1_latex)
print(f"\nSaved: {TABLES_DIR}/table1_main_results.tex")

In [ ]:
# Table 2: Ablation Study Results
ablation_for_table = {name: {'metrics': result['metrics']} 
                      for name, result in ablation_results.items()}

table2_latex = generate_latex_table(
    ablation_for_table,
    "Ablation study results showing contribution of each component.",
    "tab:ablation"
)

print("\nTable 2: Ablation Study")
print("=" * 60)
print(table2_latex)

with open(f'{TABLES_DIR}/table2_ablation.tex', 'w') as f:
    f.write(table2_latex)
print(f"\nSaved: {TABLES_DIR}/table2_ablation.tex")

In [ ]:
# Table 3: Incremental Learning Results
latex = []
latex.append(r"\begin{table}[htbp]")
latex.append(r"\centering")
latex.append(r"\caption{Performance across incremental updates.}")
latex.append(r"\label{tab:incremental}")
latex.append(r"\begin{tabular}{ccc}")
latex.append(r"\toprule")
latex.append(r"Increment & F1 Score & Recall \\")
latex.append(r"\midrule")

for m in forgetting_metrics:
    latex.append(f"{m['increment']} & {m['f1']:.4f} & {m['recall']:.4f} \\\\")

latex.append(r"\midrule")
latex.append(f"Forgetting & \\multicolumn{{2}}{{c}}{{{forgetting:.4f} ({forgetting*100:.2f}\%)}} \\\\")
latex.append(r"\bottomrule")
latex.append(r"\end{tabular}")
latex.append(r"\end{table}")

table3_latex = "\n".join(latex)

print("\nTable 3: Incremental Learning")
print("=" * 60)
print(table3_latex)

with open(f'{TABLES_DIR}/table3_incremental.tex', 'w') as f:
    f.write(table3_latex)
print(f"\nSaved: {TABLES_DIR}/table3_incremental.tex")

In [ ]:
# Table 4: Model Complexity Comparison
complexity_data = {
    'DyGAT-FR': model.count_parameters(),
    'Standard GAT': gat_model.count_parameters(),
}

latex = []
latex.append(r"\begin{table}[htbp]")
latex.append(r"\centering")
latex.append(r"\caption{Model complexity comparison.}")
latex.append(r"\label{tab:complexity}")
latex.append(r"\begin{tabular}{lcc}")
latex.append(r"\toprule")
latex.append(r"Model & Parameters & Type \\")
latex.append(r"\midrule")
latex.append(f"DyGAT-FR & {complexity_data['DyGAT-FR']:,} & Graph Neural Network \\\\")
latex.append(f"Standard GAT & {complexity_data['Standard GAT']:,} & Graph Neural Network \\\\")
latex.append(r"XGBoost & N/A & Gradient Boosting \\")
latex.append(r"LightGBM & N/A & Gradient Boosting \\")
latex.append(r"\bottomrule")
latex.append(r"\end{tabular}")
latex.append(r"\end{table}")

table4_latex = "\n".join(latex)
print("\nTable 4: Model Complexity")
print("=" * 60)
print(table4_latex)

with open(f'{TABLES_DIR}/table4_complexity.tex', 'w') as f:
    f.write(table4_latex)
print(f"\nSaved: {TABLES_DIR}/table4_complexity.tex")

## 9. Statistical Analysis

In [ ]:
# McNemar's Test for comparing classifiers
from scipy.stats import chi2

def mcnemar_test(y_true, pred1, pred2):
    """Perform McNemar's test to compare two classifiers."""
    # Build contingency table
    correct1 = (pred1 == y_true)
    correct2 = (pred2 == y_true)
    
    # b: pred1 correct, pred2 wrong
    # c: pred1 wrong, pred2 correct
    b = np.sum(correct1 & ~correct2)
    c = np.sum(~correct1 & correct2)
    
    # McNemar's statistic with continuity correction
    if b + c > 0:
        chi2_stat = (abs(b - c) - 1) ** 2 / (b + c)
        p_value = 1 - chi2.cdf(chi2_stat, df=1)
    else:
        chi2_stat = 0
        p_value = 1.0
    
    return chi2_stat, p_value, b, c

print("=" * 60)
print("STATISTICAL SIGNIFICANCE TESTS (McNemar's Test)")
print("=" * 60)

dygat_pred = all_results['DyGAT-FR (Static)']['pred']

for name, result in all_results.items():
    if name != 'DyGAT-FR (Static)':
        chi2_stat, p_value, b, c = mcnemar_test(y_test_np, dygat_pred, result['pred'])
        sig = "*" if p_value < 0.05 else ""
        sig = "**" if p_value < 0.01 else sig
        sig = "***" if p_value < 0.001 else sig
        print(f"DyGAT-FR vs {name}: χ²={chi2_stat:.4f}, p={p_value:.4f} {sig}")

In [ ]:
# Effect size calculation (Cohen's h for proportions)
def cohens_h(p1, p2):
    """Calculate Cohen's h effect size for two proportions."""
    import math
    phi1 = 2 * math.asin(math.sqrt(p1))
    phi2 = 2 * math.asin(math.sqrt(p2))
    return phi1 - phi2

print("\n" + "=" * 60)
print("EFFECT SIZE (Cohen's h) for Recall Improvement")
print("=" * 60)

dygat_recall = all_results['DyGAT-FR (Static)']['metrics']['recall']

for name, result in all_results.items():
    if name != 'DyGAT-FR (Static)':
        baseline_recall = result['metrics']['recall']
        h = cohens_h(dygat_recall, baseline_recall)
        
        # Interpret effect size
        if abs(h) < 0.2:
            interpretation = "small"
        elif abs(h) < 0.5:
            interpretation = "medium"
        else:
            interpretation = "large"
        
        print(f"vs {name}: h={h:.4f} ({interpretation})")

## 10. Export Results

In [ ]:
# Compile all results into a single JSON file
export_data = {
    'experiment_info': {
        'date': datetime.now().isoformat(),
        'dataset': 'Synthetic' if USE_SYNTHETIC else 'UNSW-NB15',
        'device': str(DEVICE),
        'seed': SEED
    },
    'model_config': MODEL_CONFIG,
    'training_config': TRAINING_CONFIG,
    'main_results': {name: result['metrics'] for name, result in all_results.items()},
    'ablation_results': {name: result['metrics'] for name, result in ablation_results.items()},
    'incremental_results': forgetting_metrics,
    'forgetting_metric': forgetting,
    'model_parameters': model.count_parameters()
}

# Save JSON
results_json_path = f'{OUTPUT_DIR}/all_results.json'
with open(results_json_path, 'w') as f:
    json.dump(export_data, f, indent=2, default=str)

print(f"All results exported to {results_json_path}")

In [ ]:
# Create results summary CSV
summary_rows = []

for name, result in all_results.items():
    row = {'Model': name, **result['metrics']}
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
summary_csv_path = f'{OUTPUT_DIR}/results_summary.csv'
summary_df.to_csv(summary_csv_path, index=False)

print(f"Results summary saved to {summary_csv_path}")
print("\n" + summary_df.to_string(index=False))

In [ ]:
# List all generated artifacts
print("\n" + "=" * 60)
print("GENERATED ARTIFACTS FOR JOURNAL PAPER")
print("=" * 60)

print("\nFigures:")
for f in sorted(os.listdir(FIGURES_DIR)):
    print(f"  - {FIGURES_DIR}/{f}")

print("\nTables (LaTeX):")
for f in sorted(os.listdir(TABLES_DIR)):
    print(f"  - {TABLES_DIR}/{f}")

print("\nCheckpoints:")
for f in sorted(os.listdir(CHECKPOINTS_DIR)):
    print(f"  - {CHECKPOINTS_DIR}/{f}")

print("\nData Files:")
print(f"  - {results_json_path}")
print(f"  - {summary_csv_path}")

In [ ]:
# Download results (Colab only)
if IN_COLAB:
    import shutil
    
    # Zip results folder
    shutil.make_archive('journal_results', 'zip', OUTPUT_DIR)
    
    # Download
    from google.colab import files
    files.download('journal_results.zip')
    
    print("\nResults downloaded as journal_results.zip")
else:
    print(f"\nResults saved locally in: {OUTPUT_DIR}")

## Summary

This notebook has generated all artifacts required for the DyGAT-FR journal paper:

### Figures
1. **fig1_training_curves.pdf** - Training loss, F1, and recall curves
2. **fig2_roc_pr_curves.pdf** - ROC and Precision-Recall curves comparison
3. **fig3_confusion_matrices.pdf** - Confusion matrices for all models
4. **fig4_ablation_study.pdf** - Ablation study bar chart
5. **fig5_incremental_performance.pdf** - Performance across increments
6. **fig6_model_comparison.pdf** - Overall model comparison

### Tables (LaTeX)
1. **table1_main_results.tex** - Main comparison results
2. **table2_ablation.tex** - Ablation study results
3. **table3_incremental.tex** - Incremental learning results
4. **table4_complexity.tex** - Model complexity comparison

### Statistical Analysis
- McNemar's test for classifier comparison
- Cohen's h effect size for recall improvement

### Model Checkpoints
- Static trained model
- Incrementally trained model

---

**Next Steps:**
1. Run on full UNSW-NB15 dataset (set `USE_SYNTHETIC = False`)
2. Add additional datasets (CIC-IDS2017, ToN-IoT)
3. Run multiple seeds for confidence intervals
4. Generate architecture diagram